### What this script does

This script processes 10-minute merged mast, radiometer, and sonic data to compute turbulent momentum fluxes and update each daily merged_data_10min.csv file with new stress components.

It automatically:

- Loops over all target months (2024-03, 2024-04, 2024-05) and their valid dates.

- Searches each day’s folder inside your local data directory for:

    - merged_data_10min.csv

    - Raw .dat files containing high-frequency wind components (Ux, Uy, Uz).

    - Rotates the wind components by a fixed theta angle (116°) to align them with the mean wind direction.

    - Divides each .dat file into ~10-minute chunks (12 000 samples each), and for each chunk computes:

        - uw_flux_corr → covariance of u′ and w′

        - vw_flux_corr → covariance of v′ and w′

        - uv_flux_corr → covariance of u′ and v′
        (these represent corrected turbulent fluxes).

- Merges these fluxes into the corresponding daily merged_data_10min.csv.

- Computes corrected stress components using air density (rho_air_Tv):

    - tau_xz_corr = −ρ * uw_flux_corr

    - tau_yz_corr = −ρ * vw_flux_corr

    - tau_xy_corr = −ρ * uv_flux_corr

- Saves the updated CSV back to disk.

Edit before running:
Base directory containing the 10-min merged data
base_dir = r"C:\path\to\your\Sonic"


In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# Base directory containing the 10 min-merged mast/radiometer/sonic data
# ⚠️ Edit this path to match your local setup before running.
# Example: base_dir = r"D:\MyThesis\data\Sonic"
base_dir = r"C:\path\to\your\Sonic"

months = ['2024-03', '2024-04', '2024-05']
theta_deg = 116
theta_rad = np.deg2rad(theta_deg)

for month_str in months:
    month_dt = datetime.strptime(month_str, '%Y-%m')
    for day in range(1, 32):
        try:
            date_dt = month_dt.replace(day=day)
            date_str = date_dt.strftime('%Y-%m-%d')  # '2024-05-01

        except ValueError:
            continue  # skip invalid dates

        data_dir = os.path.join(base_dir, month_str, date_str)
        
        merged_file_path = os.path.join(data_dir, 'merged_data_10min.csv')
        print(f"Checking {data_dir}")

        if not os.path.exists(merged_file_path):
            continue

        if not any(fname.endswith('.dat') for fname in os.listdir(data_dir)):
            continue

        timestamps_10min = []
        uw_flux_10min_corr = []
        vw_flux_10min_corr = []
        uv_flux_10min_corr = []

        for filename in os.listdir(data_dir):
            if filename.endswith('.dat'):
                file_path = os.path.join(data_dir, filename)
                data = pd.read_csv(file_path, skiprows=1, delimiter=',', encoding='latin1', low_memory=False)
                data.dropna(inplace=True)

                data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], errors='coerce')
                for var in ['Ux', 'Uy', 'Uz']:
                    data[var] = pd.to_numeric(data[var], errors='coerce')

                data['Ux_corrected'] = data['Ux'] * np.cos(theta_rad) + data['Uy'] * np.sin(theta_rad)
                data['Uy_corrected'] = -data['Ux'] * np.sin(theta_rad) + data['Uy'] * np.cos(theta_rad)

                for i in range(0, len(data), 12000):
                    chunk = data.iloc[i:i+12000]
                    if chunk.empty:
                        continue

                    u_mean = chunk['Ux_corrected'].mean()
                    v_mean = chunk['Uy_corrected'].mean()
                    w_mean = chunk['Uz'].mean()

                    u_prime = chunk['Ux_corrected'] - u_mean
                    v_prime = chunk['Uy_corrected'] - v_mean
                    w_prime = chunk['Uz'] - w_mean

                    uw_flux_corr = (u_prime * w_prime).mean()
                    vw_flux_corr = (v_prime * w_prime).mean()
                    uv_flux_corr = (u_prime * v_prime).mean()

                    timestamps_10min.append(chunk['TIMESTAMP'].iloc[0].floor('10T'))
                    uw_flux_10min_corr.append(uw_flux_corr)
                    vw_flux_10min_corr.append(vw_flux_corr)
                    uv_flux_10min_corr.append(uv_flux_corr)

        if not timestamps_10min:
            continue

        flux_df = pd.DataFrame({
            'TIMESTAMP': timestamps_10min,
            'uw_flux_corr': uw_flux_10min_corr,
            'vw_flux_corr': vw_flux_10min_corr,
            'uv_flux_corr': uv_flux_10min_corr,
        })

        try:
            merged_df = pd.read_csv(merged_file_path, parse_dates=['TIMESTAMP'])
            merged_df = pd.merge(merged_df, flux_df, on='TIMESTAMP', how='left')

            # Compute corrected stress components
            merged_df['tau_xz_corr'] = -merged_df['rho_air_Tv'] * merged_df['uw_flux_corr']
            merged_df['tau_yz_corr'] = -merged_df['rho_air_Tv'] * merged_df['vw_flux_corr']
            merged_df['tau_xy_corr'] = -merged_df['rho_air_Tv'] * merged_df['uv_flux_corr']

            merged_df.to_csv(merged_file_path, index=False)
            print(f'Updated: {merged_file_path}')
        except Exception as e:
            print(f"Error processing {merged_file_path}: {e}")